In [17]:
import pandas as pd

# 读取CSV文件
file_path = '/workspaces/SFP/Training_data/data_13_17_1_std.csv'  # 请替换为你的CSV文件路径
df = pd.read_csv(file_path)

# 将 'postime' 列解析为日期时间格式
df['postime'] = pd.to_datetime(df['postime'], format='%Y-%m-%d %H:%M:%S')

# 提取日期和小时
df['date'] = df['postime'].dt.date
df['hour'] = df['postime'].dt.hour

# 初始化一个空的 DataFrame 用于存储结果
filtered_df = pd.DataFrame()

# 按 num（船编号）分组
for num, group in df.groupby('num'):
    # 检查每个日期是否包含完整的 0-23 小时
    complete_dates = group.groupby('date')['hour'].apply(lambda x: len(set(x)) == 24 and set(x) == set(range(24)))
    valid_dates = complete_dates[complete_dates].index  # 提取满足条件的日期

    # 筛选出完整日期的数据
    filtered_group = group[group['date'].isin(valid_dates)]

    # 将筛选后的数据添加到结果 DataFrame
    filtered_df = pd.concat([filtered_df, filtered_group], ignore_index=True)

# 删除辅助列
filtered_df = filtered_df.drop(columns=['date', 'hour'])

# 保存结果为新的CSV文件
output_file = '/workspaces/SFP/validation_data/filtered_data.csv'
filtered_df.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"过滤后的数据已保存到: {output_file}")


过滤后的数据已保存到: /workspaces/SFP/validation_data/filtered_data.csv


# Validation for fuel consumption

In [1]:
import pandas as pd
import pickle
from sklearn.preprocessing import StandardScaler
from class_calm_water_resistance_estimatoin import *


def Rtotal2Pe(row):
    """"

    每一行反算R_total函数

    
    """
    cu = row['stream_val']*np.sin(np.deg2rad(row['stream_direction']))
    cv = row['stream_val']*np.cos(np.deg2rad(row['stream_direction']))
    sog = row['SOG']*0.5144 # knot to m/s
    r_total = row['R_t_pre']
    heading_ship = row['heading']
    V_water = speedGPS2Water(sog, heading_ship, cu, cv) 
    Pe = V_water*r_total

    return Pe # 单位千牛


def calFuelHour(row):
    etaR = 1.0 # 准确计算较复杂，但整体波动小，经验值是1-1.07或者0.98
    etaO = 0.60 # 可以算，但需要知道V_A和thruster force T的具体意义，经验值0.55-0.7
    etaS = 0.99 # 无计算公式，取经验值，0.99，0.98，0.95-0.96三种情况
    etaH = 1.1 # 要计算t, w，1-t/1-w,需要知道力和速度的关系。
    SOFC = 200 # g/kwh
    eta = etaR*etaO*etaS*etaH

    P = row['P_pre']
    fuel_hour = P/eta*SOFC/1000000

    return fuel_hour

# 读取 CSV 文件
file_path = '/workspaces/SFP/validation_data/filtered_data.csv'  # 替换为你的 CSV 文件路径
df = pd.read_csv(file_path)

# 提取指定列
columns_to_use = [
    'SOG', 'heading', 'draught', 'wind_val', 'wind_direction',
    'wave_val', 'wave_direction', 'stream_val', 'stream_direction'
]
input_df = df[columns_to_use]

# 加载预训练的集成模型
with open('ensemble_model.pkl', 'rb') as file:
    ensemble_model = pickle.load(file)

# 数据预处理（标准化）
feature_scaler = StandardScaler()
input_features_scaled = feature_scaler.fit_transform(input_df)

# 进行预测
df['R_a_pre'] = ensemble_model.predict(input_features_scaled)

# 计算 R_t_pre 列：R_a_pre 和 R_calm 的和
df['R_t_pre'] = df['R_a_pre'] + df['R_calm']

# 计算 P_pre 列
df['P_pre'] = df.apply(Rtotal2Pe, axis=1)

# 计算 P_pre 列
df['Fuel_hour'] = df.apply(calFuelHour, axis=1)

# 提取日期（去掉时间部分）
df['date'] = pd.to_datetime(df['postime']).dt.date

# 按日期汇总 Fuel_hour
# 按 num 和 date 分组计算每天的 Fuel_hour
grouped = df.groupby(['num', 'date']).agg(
    Fuel_hour=('Fuel_hour', 'sum')  # 汇总每天的 Fuel_hour
).reset_index()

# 按 num 排序，然后在每个 num 内部按日期排序
grouped = grouped.sort_values(by=['num', 'date']).reset_index(drop=True)


grouped.to_csv('/workspaces/SFP/validation_data/predicted_day.csv')
# 保存结果为新的 CSV 文件
# output_file = 'daily_fuel_hour.csv'
# daily_fuel_hour.to_csv(output_file, index=False, encoding='utf-8-sig')

# print(f"统计完成，每日的燃料消耗量已保存到: {output_file}")

# # 保存结果为新的 CSV 文件
# output_file = '/workspaces/SFP/validation_data/predicted.csv'
# df.to_csv(output_file, index=False, encoding='utf-8-sig')

# print(f"预测完成，结果已保存到: {output_file}")

/home/codespace/.python/current/lib/python3.12/site-packages/xgboost/core.py:158: UserWarning: [13:01:59] WARNING: /workspace/src/collective/../data/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)
/home/codespace/.python/current/lib/python3.12/site-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator VotingRegressor from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-

In [3]:
df_fuel = pd.read_csv('/workspaces/SFP/raw_data/mrv.csv')


# 确保日期格式一致
grouped['date'] = pd.to_datetime(grouped['date']).dt.date  # 保留日期部分
df_fuel['input_date'] = pd.to_datetime(df_fuel['input_date']).dt.date  # 保留日期部分

# 合并数据，按 'num' 和日期匹配
merged_df = pd.merge(
    grouped, 
    df_fuel, 
    left_on=['num', 'date'], 
    right_on=['num', 'input_date'], 
    how='inner'
)

# 删除重复的日期列，只保留一个
merged_df = merged_df.rename(columns={'date': 'merged_date', 'Fuel_hour': 'Predict_fuel'})
final_df = merged_df[['num', 'merged_date', 'Predict_fuel', 'daily_fuel_mrv']]


final_df.to_csv('/workspaces/SFP/validation_data/final_results.csv')